In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Load data
import pandas as pd
data=pd.read_table('/content/drive/MyDrive/ML/input_file/finalexam/kor.txt')
data

,Go.,가.,CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8363271 (Eunhee)
0,Hi.,안녕.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Run!,뛰어!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Run.,뛰어.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
3,Who?,누구?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Wow!,우와!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
...,...,...,...
5886,I started a new blog. I'll do my best not to b...,난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되지 ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
5887,I think it's a shame that some foreign languag...,몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 있었...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
5888,And the good news is that today the economy is...,"다음으로 좋은 소식은 오늘 경제가 재성장한다는 것입니다. 임금, 소득, 집값, 퇴직...",CC-BY 2.0 (France) Attribution: tatoeba.org #5...
5889,If someone who doesn't know your background sa...,만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 원어...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


Because the convert file is not compatible to Korean, the Korean can be shown differnt keywords. Thus I upload with my ipynb file.

In [ ]:
# ram crash happened if I use all data
# So I use a part of total data
X=data.iloc[:2000,0]
Y=data.iloc[:2000,1]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

# Toy dataset for English to Korean translation
input_texts = X
target_texts = Y
print(X[:10])
print(Y[:10])

0      Hi.
1     Run!
2     Run.
3     Who?
4     Wow!
5    Duck!
6    Fire!
7    Help!
8    Hide.
9    Jump!
Name: Go., dtype: object
0     안녕.
1     뛰어!
2     뛰어.
3     누구?
4     우와!
5     숙여!
6      쏴!
7    도와줘!
8     숨어.
9     점프!
Name: 가., dtype: object


In [ ]:
input_vocab = set(char for word in input_texts for char in word)
target_vocab = set(char for word in target_texts for char in word)
#print(input_vocab)
#print(target_vocab)

input_vocab_size = len(input_vocab)
target_vocab_size = len(target_vocab)
print(input_vocab_size)
print(target_vocab_size)

input_max_len = max(len(word) for word in input_texts)
target_max_len = max(len(word) for word in target_texts) + 1  # +1 for the end token
print(input_max_len)
print(target_max_len)

65
711
21
23


In [ ]:
# Create mapping from characters to indices and vice versa
input_char_index = dict((char, i) for i, char in enumerate(input_vocab))
target_char_index = dict((char, i) for i, char in enumerate(target_vocab))
reverse_input_char_index = dict((i, char) for i, char in enumerate(input_vocab))
reverse_target_char_index = dict((i, char) for i, char in enumerate(target_vocab))

# Prepare the data
encoder_input_data = np.zeros((len(input_texts), input_max_len, input_vocab_size), dtype='float32')    #that is, size(2000,21,65)
decoder_input_data = np.zeros((len(input_texts), target_max_len, target_vocab_size), dtype='float32')  #that is, size(2000,23,711)
decoder_target_data = np.zeros((len(input_texts), target_max_len, target_vocab_size), dtype='float32') #that is, size(2000,23,711)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_char_index[char]] = 1.0                   # set component of encoder input data which has char 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_char_index[char]] = 1.0                  # set component of decoder input data which has char 1
        if t > 0:   # for nonzero t
            decoder_target_data[i, t - 1, target_char_index[char]] = 1.0         # set previous component of decoder input data which has char 1


In [ ]:
print(input_char_index['Y'])
print(target_char_index['좁'])

57
661


In [ ]:
# Define the Seq2Seq model
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None, input_vocab_size))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)   #set hidden state and others
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, target_vocab_size))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)  #recieve the context from encoder
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)               # seq2seq

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, epochs=800, batch_size=20)
#epochs is set 20 despite low accuracy

Epoch 1/800
100/100 [==============================] - 15s 147ms/step - loss: 1.1659 - accuracy: 0.1560
Epoch 2/800
100/100 [==============================] - 15s 152ms/step - loss: 1.1055 - accuracy: 0.1635
Epoch 3/800
100/100 [==============================] - 15s 153ms/step - loss: 1.0700 - accuracy: 0.1668
Epoch 4/800
100/100 [==============================] - 15s 147ms/step - loss: 1.0338 - accuracy: 0.1725
Epoch 5/800
100/100 [==============================] - 15s 146ms/step - loss: 1.0085 - accuracy: 0.1757
Epoch 6/800
100/100 [==============================] - 15s 147ms/step - loss: 0.9863 - accuracy: 0.1787
Epoch 7/800
100/100 [==============================] - 15s 147ms/step - loss: 0.9707 - accuracy: 0.1805
Epoch 8/800
100/100 [==============================] - 15s 147ms/step - loss: 0.9579 - accuracy: 0.1832
Epoch 9/800
100/100 [==============================] - 15s 147ms/step - loss: 0.9421 - accuracy: 0.1850
Epoch 10/800
100/100 [==============================] - 15s 147m

In [ ]:
# Inference setup
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))           # h input state has(256,1)shape
decoder_state_input_c = Input(shape=(latent_dim,))           # c input state has(256,1)shape
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [ ]:
# Function to decode sequences
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, target_vocab_size))
    # Populate the first character of target sequence with the start character
    target_seq[0, 0, target_char_index[' ']] = 1.0

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find the stop character
        if sampled_char == '\n' or len(decoded_sentence) > target_max_len:
            stop_condition = True

        # Update the target sequence (length 1)
        target_seq = np.zeros((1, 1, target_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Test the model with first 10 input sequences
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(f'Input: {input_texts[seq_index]}')
    print(f'Decoded: {decoded_sentence}')
    #print('---')

1/1 [==============================] - 0s 41ms/step
Input: Hi.
Decoded: 즐튼!.....................
1/1 [==============================] - 0s 26ms/step
Input: Run!
Decoded: 좋닷......................
1/1 [==============================] - 0s 30ms/step
Input: Run.
Decoded: 짖네......................
1/1 [==============================] - 0s 28ms/step
Input: Who?
Decoded: 즐튼!.....................
1/1 [==============================] - 0s 23ms/step
Input: Wow!
Decoded: 즐긴세요....................
1/1 [==============================] - 0s 22ms/step
Input: Duck!
Decoded: 좋닷!.....................
1/1 [==============================] - 0s 35ms/step
Input: Fire!
Decoded: 끄러워?....................
1/1 [==============================] - 0s 23ms/step
Input: Help!
Decoded: 쏜다......................
1/1 [==============================] - 0s 20ms/step
Input: Hide.
Decoded: 닷.......................
1/1 [==============================] - 0s 21ms/step
Input: Jump!
Decoded: 즐튼!.....................
1/1 [===========

Since the traing result is poor, the test is terrible